In [1]:
import numpy as np
from scipy import ndimage as ndi
import pandas as pd
from matplotlib import pyplot as plt
from skimage.morphology import skeletonize_3d
from skimage.filters import threshold_multiotsu
from skimage.measure import label, regionprops_table
from skimage.exposure import equalize_adapthist
import imageio.v2 as imageio
import glob
import os

# Load Data
change the data_dir values for the folder where tiff images stored

In [2]:
%%time
#dir_name is the director to store all the tmp and final results 
data_dir = "demo_data"
#The inital index of data is z,y,x
imagelist = [imageio.imread(file) for file in glob.glob(os.path.join(data_dir,"*[0-9][0-9][0-9].tiff"))]
data = np.array(imagelist)
del(imagelist)
print("initial data dimensions:", data.shape)
#change the index of data to x,y,z, and change the matrix from view to continous stored array
data = np.ascontiguousarray(np.moveaxis(data,source=[0,2],destination=[2,0]))
print("tunned data dimensions:", data.shape)

initial data dimensions: (361, 548, 364)
tunned data dimensions: (364, 548, 361)
CPU times: total: 2.28 s
Wall time: 5.2 s


# Threshold and Skeletonization

In [3]:
%%time
#binarization
data = equalize_adapthist(data)
thresholds = threshold_multiotsu(data)
mask = ndi.binary_fill_holes((np.digitize(data, thresholds) == 2)).astype(np.uint8)
mask = ndi.binary_opening(input=mask,structure=np.ones([3,3,3],dtype=int))

CPU times: total: 45.8 s
Wall time: 45.8 s


In [4]:
%%time
#skeletonization
skeleton = skeletonize_3d(mask)
skeleton[skeleton.nonzero()]=1

CPU times: total: 9.98 s
Wall time: 10 s


# Trace the fibers based on the orientation field calculated in step-1

In [5]:
%%time
#culculate the neibours of each points
from skimage.morphology import cube
neigbours = ndi.convolve(skeleton,cube(3),mode='constant',cval=0)
print("cross point amounts: ",np.sum((neigbours>=4) & skeleton))

cross point amounts:  25332
CPU times: total: 4 s
Wall time: 4 s


In [6]:
#load the orientation field, calculated by step1
px = np.load(os.path.join(data_dir,"px.npy"))
py = np.load(os.path.join(data_dir,"py.npy"))
pz = np.load(os.path.join(data_dir,"pz.npy"))
correlation = np.load(os.path.join(data_dir,"correlation.npy"))

In [8]:
%%time
#cut the branches based on orientation field
for points_index,(x,y,z) in enumerate(zip(*np.where((neigbours>=4) & skeleton))):
    if points_index%10000 == 0:
        print("Already treat points amouts:", points_index)
    #remove the cross points in the boundery
    if (x <= 1) | (y <= 1) | (z <= 1) | (x >= skeleton.shape[0]-2) | (y >= skeleton.shape[1]-2) | (z >= skeleton.shape[2]-2):
        skeleton[x,y,z] == 0
        continue
    #recalculate the neighbors of this points
    cube3_ref = skeleton[x-1:x+2,y-1:y+2,z-1:z+2]# numpy is default pass by fererence
    cube3_copy = np.copy(cube3_ref)
    cube3_copy[1,1,1] = 0
    connectivity = np.sum(cube3_copy)
    if connectivity <= 2:
        continue
    #store all the branches information in the pandas table
    branches_df = pd.DataFrame(cube3_copy.nonzero(),index=["bx","by","bz"]).T
    #read the orientation information from the orientation field
    orient_vec = np.array([px[x,y,z],py[x,y,z],pz[x,y,z]])
    #calculate the correlations of all the branches with the orientation field
    for ind, row in branches_df.iterrows():
        bx = int(row["bx"])
        by = int(row["by"])
        bz = int(row["bz"])

        cube3_branch = np.copy(skeleton[x+bx-2:x+bx+1,y+by-2:y+by+1,z+bz-2:z+bz+1])
        cube3_branch[1,1,1] = 0
        cube3_branch[-bx,-by,-bz] = 0
        #find if any mother branches point in this cube, if any, set it to zero
        tmp_df = branches_df.loc[:,["bx","by","bz"]]-[bx,by,bz]
        tmp_df = tmp_df[(tmp_df["bx"] >= -1) & (tmp_df["bx"] <= 1) & (tmp_df["by"] >= -1) & (tmp_df["by"] <= 1) & (tmp_df["bz"] >= -1) & (tmp_df["bz"] <= 1)]
        if len(tmp_df) > 1:
            cube3_branch[tmp_df["bx"]+1,tmp_df["by"]+1,tmp_df["bz"]+1] = 0

        #store all the points in this branch in this numpy array, rows are (x,y,z) respectively
        #the relative value to the center point of x,y,z
        branch_points = np.array(cube3_branch.nonzero())
        branch_points[0,:] += bx-2
        branch_points[1,:] += by-2
        branch_points[2,:] += bz-2
        branch_points = np.c_[branch_points,[bx-1,by-1,bz-1]]
        #calculate the orientation vector
        orient_vec_b = np.sum(branch_points,axis=1).astype(np.float32)
        orient_vec_b /= np.linalg.norm(orient_vec_b)
        #update the remained 2 branches
        branches_df.loc[ind,"correlation"] = np.abs(np.dot(orient_vec_b,orient_vec))
    #sort the results by correlations,
    branches_df = branches_df.sort_values(by = "correlation",ascending=False)
    #remove all the branches with lower correlations, only kept the largest correlated two branches
    d_df = branches_df.iloc[2:]
    cube3_ref[d_df["bx"],d_df["by"],d_df["bz"]] = 0

Already treat points amouts: 0
Already treat points amouts: 10000
Already treat points amouts: 20000
CPU times: total: 4min 58s
Wall time: 4min 59s


In [9]:
%%time
#check by re-culculating the neibours of each points
from skimage.morphology import cube
neigbours = ndi.convolve(skeleton,cube(3),mode='constant',cval=0)
print("cross point amounts: ",np.sum((neigbours>=4) & skeleton))

cross point amounts:  8
CPU times: total: 3.19 s
Wall time: 3.19 s


# label and calculate properties

In [10]:
#remove small objects, and label
from skimage.morphology import remove_small_objects, label
skeleton_final = remove_small_objects(skeleton.astype(bool),min_size=30,connectivity=3)
label_skeleton = label(label_image=skeleton_final, connectivity=3)
np.save(os.path.join(data_dir,"skeleton_final.npy"),skeleton_final)


In [ ]:
#visualized with napari package
import napari
napari.view_labels(ndi.grey_dilation(label_skeleton,size=3))

In [13]:
%%time
#calculate properties using pandas package
skeleton_df = pd.DataFrame(np.array(label_skeleton.nonzero()).T,columns=["x","y","z"])
skeleton_df["label"] = label_skeleton[skeleton_df["x"],skeleton_df["y"],skeleton_df["z"]]
regions_df = pd.DataFrame(regionprops_table(label_skeleton,properties=('label', 'area',"centroid")))
regions_df = regions_df.set_index("label")
#skeleton_df = pd.merge(left=skeleton_df,right=regions_df,how="inner",on=["label"])
#calculate orientation of individual fiber by cov ops 
cov_df = skeleton_df.groupby(by = "label").cov()
for label_ind, row in regions_df.iterrows():
    cov_mat = cov_df.loc[label_ind,:]
    eig_value, eig_mat = np.linalg.eig(cov_mat)
    eig_vec = eig_mat[:,np.argmax(eig_value)]
    regions_df.loc[label_ind,"px"] = eig_vec[0]
    regions_df.loc[label_ind,"py"] = eig_vec[1]
    regions_df.loc[label_ind,"pz"] = eig_vec[2]
#calculate the orientation tensor of glass fibers
regions_df["Axx"] = regions_df["px"]**2
regions_df["Axy"] = regions_df["px"]*regions_df["py"]
regions_df["Axz"] = regions_df["px"]*regions_df["pz"]
regions_df["Ayy"] = regions_df["py"]**2
regions_df["Ayz"] = regions_df["py"]*regions_df["pz"]
regions_df["Azz"] = regions_df["pz"]**2
#save data
with pd.HDFStore(os.path.join(data_dir,"DataFrame.h5"),mode="a") as store:
    store["regions_df"] = regions_df
    store["skeleton_df"] = skeleton_df
regions_df

CPU times: total: 12.8 s
Wall time: 12.9 s


,area,centroid-0,centroid-1,centroid-2,px,py,pz,Axx,Axy,Axz,Ayy,Ayz,Azz
label,,,,,,,,,,,,,
1,170,84.500000,109.894118,239.405882,0.998583,-0.052106,-0.010772,0.997169,-0.052033,-0.010757,0.002715,0.000561,0.000116
2,55,27.000000,167.745455,52.072727,0.925267,-0.114783,0.361534,0.856118,-0.106205,0.334515,0.013175,-0.041498,0.130707
3,86,42.500000,180.488372,248.500000,0.894597,0.026507,0.446087,0.800303,0.023713,0.399068,0.000703,0.011825,0.198994
4,30,14.500000,210.733333,327.133333,0.969459,0.244864,0.013820,0.939850,0.237386,0.013398,0.059959,0.003384,0.000191
5,51,24.862745,236.000000,306.196078,-0.924030,-0.301151,-0.235533,0.853832,0.278273,0.217640,0.090692,0.070931,0.055476
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4668,46,360.695652,49.413043,144.500000,-0.086173,-0.032756,-0.995742,0.007426,0.002823,0.085806,0.001073,0.032617,0.991501
4669,120,360.050000,65.441667,141.500000,0.034740,-0.064507,0.997312,0.001207,-0.002241,0.034647,0.004161,-0.064333,0.994632
4670,31,360.161290,144.483871,276.000000,-0.151149,-0.107032,-0.982699,0.022846,0.016178,0.148534,0.011456,0.105181,0.965698


In [12]:
#print average orientation tensor A
print("The Orientation Tensor is:")
[[regions_df.Axx.mean(), regions_df.Axy.mean(), regions_df.Axz.mean()], \
[regions_df.Axy.mean(), regions_df.Ayy.mean(), regions_df.Ayz.mean(),], \
[regions_df.Axz.mean(), regions_df.Ayz.mean(), regions_df.Azz.mean()]]

The Orientation Tensor is:


[[0.4468475557106242, -0.007560628929769429, -0.01970903545844713],
 [-0.007560628929769429, 0.05442863296376422, 0.015064860608150706],
 [-0.01970903545844713, 0.015064860608150706, 0.49872381132561155]]

# Done
Furthure more: 

pixel wise segmentation could be applied by watershed segmentation method, with using the individual lines obtained in this step as seeds 